In [14]:
import numpy as np

# Constants
n_iterations = 100  # number of iterations
n_device = [50, 100, 150, 200, 250, 300]  # number of devices
n_mecs = 10  # number of MECs
mec_capacity = 4  # MEC computing capacity (GHz)
transmission_power = 0.5  # Transmission power (W)

# Voltages (assumed constants)
mec_voltage = 1.0  # Assumed voltage for MEC
device_voltage = 1.0  # Assumed voltage for device

# Task data (Mbit)
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45])
transmission_rate = np.array([500, 1000, 1500, 2000, 2500, 3000]) * 1e-3  # Convert to Mbit/s

def fitness(position, device_capacity, td, tr):
    transmission_time = td / tr
    computation_time = td / (position * mec_capacity + (1 - position) * device_capacity) * n / 60

    energy_transmission = n * transmission_power * transmission_time  # energy consumed during transmission
    energy_computation_mec = n * position * mec_capacity * mec_voltage ** 2
    energy_computation_device = n * (1 - position) * device_capacity * device_voltage ** 2

    total_energy_consumption = energy_transmission + energy_computation_mec + energy_computation_device
    cost = transmission_time + computation_time
    mean_completion_time = cost  # Since the number of tasks is 1 per particle, total time is mean time

    return cost, total_energy_consumption, mean_completion_time


for n in n_device:
    n_particles = n * n_mecs
    device_capacity = np.random.uniform(0.5, 1, n_particles)
    task_data_per_particle = np.random.choice(task_data, n_particles)
    transmission_rate_per_particle = np.random.choice(transmission_rate, n_particles)

    particles = np.random.uniform(0, 1, size=n_particles)
    pbest_positions = particles.copy()
    pbest_fitness_value = np.full(n_particles, float('inf'))

    gbest_fitness_value = float('inf')
    gbest_position = 0

    for i in range(n_iterations):
        fitness_values, energy_consumptions, mean_completion_times = np.vectorize(fitness)(particles, device_capacity, task_data_per_particle, transmission_rate_per_particle)

        # Update pbest and gbest
        update_pbest = fitness_values < pbest_fitness_value
        pbest_positions[update_pbest] = particles[update_pbest]
        pbest_fitness_value[update_pbest] = fitness_values[update_pbest]

        min_fitness_idx = np.argmin(fitness_values)
        if fitness_values[min_fitness_idx] < gbest_fitness_value:
            gbest_fitness_value = fitness_values[min_fitness_idx]
            gbest_position = particles[min_fitness_idx]
            gbest_energy_consumption = energy_consumptions[min_fitness_idx]
            gbest_mean_completion_time = mean_completion_times[min_fitness_idx]

        # Update particles' positions
        particles = pbest_positions + 0.5 * np.random.rand(n_particles) * (gbest_position - pbest_positions)
        particles = np.random.uniform(0, 1, size=n_particles)

    print(f"For {n} devices, the best position is {gbest_position}, with energy consumption {gbest_energy_consumption}, "
          f"and mean completion time {gbest_mean_completion_time} in iteration number {n_iterations}")


For 50 devices, the best position is 0.9984927910725686, with energy consumption 241.4397204776988, and mean completion time 2.709516687522825 in iteration number 100
For 100 devices, the best position is 0.9996747051759948, with energy consumption 483.2321814492784, and mean completion time 3.7505269659884597 in iteration number 100
For 150 devices, the best position is 0.9997808313678386, with energy consumption 724.8935210495968, and mean completion time 4.792221342968935 in iteration number 100
For 200 devices, the best position is 0.9998096840770895, with energy consumption 966.5523792446112, and mean completion time 5.833928665371911 in iteration number 100
For 250 devices, the best position is 0.9994669213530033, with energy consumption 1207.8729800183419, and mean completion time 6.8773987778008925 in iteration number 100
For 300 devices, the best position is 0.9993374106841827, with energy consumption 1449.3539501994455, and mean completion time 7.920033321900341 in iteration 